# Exercise 1: Patients Prescribed Opioids

## Introduction

This notebook explores a FHIR server with a RESTful API which contains data with patients currently prescribed opioids.

### Environment Configuration

First, let's configure the environment with the libraries and settings that will be used throughtout the rest of the exercise.

In [24]:
import json
import requests
import pandas as pd
from IPython.display import Markdown, display

def print_md(string):
    display(Markdown(string))

s = requests.Session()
s.headers.update({'Accept':'application/fhir+json', 'Content-Type': 'application/fhir+json'})
s.verify = False
requests.packages.urllib3.disable_warnings()

FHIR_SERVER = 'https://api.logicahealth.org/opioids/open'

## Querying the Server

All servers are required to support the `capabilities` interaction which documents the server's functionality. The capability interaction is of the form:

`GET [base]/metadata`

📘[Read more about the capabilities interaction](https://www.hl7.org/fhir/http.html#capabilities)

In [9]:
r = s.get(f"{FHIR_SERVER}/metadata")
print(f"HTTP Status Code: {r.status_code}")
capability_statement = r.json()

HTTP Status Code: 200


The HTTP status code, `200 - OK` indicates a successful response. The CapabilityStatement is usually a very large resource so we'll focus on a few key elements.

In [22]:
print(f"FHIR Version Supported: {capability_statement['fhirVersion']}")
print(f"Formats Supported: {', '.join(capability_statement['format'])}")
print()

print(f"Number of RESTful Endpoints: {len(capability_statement['rest'])}")
for rest in capability_statement['rest']:
    print(f"Endpoint Type: {rest['mode']}")

FHIR Version Supported: 4.0.1
Formats Supported: application/fhir+xml, application/fhir+json

Number of RESTful Endpoints: 1
Endpoint Type: server


There is only a single server RESTful endpoint. We can also see the types of FHIR Resources available on this endpoint and the supported operations. Keep in mind that there might not be any data available for a particular FHIR resource even if the server supports that type.

In [58]:
rest = capability_statement['rest'][0]
data = {'Resource': [resource['type'] for resource in rest['resource']],
        'Interaction': [[interaction['code'] for interaction in resource['interaction']] for resource in rest['resource']],
        'Search': [[search['name'] for search in resource['searchParam']] for resource in rest['resource']]}

capabilities_df = pd.DataFrame.from_dict(data)

In [59]:
print('Supported Resources')
display(capabilities_df[['Resource']])

print('Interactions Supported by Resource')
display(capabilities_df.explode('Interaction')[['Resource','Interaction']])

print('Searches Supported by Resource')
display(capabilities_df.explode('Search')[['Resource','Search']])

Supported Resources


,Resource
0,Account
1,ActivityDefinition
2,AdverseEvent
3,AllergyIntolerance
4,Appointment
...,...
141,TestReport
142,TestScript
143,ValueSet
144,VerificationResult


Interactions Supported by Resource


,Resource,Interaction
0,Account,read
0,Account,vread
0,Account,update
0,Account,patch
0,Account,delete
...,...,...
145,VisionPrescription,delete
145,VisionPrescription,history-instance
145,VisionPrescription,history-type
145,VisionPrescription,create


Searches Supported by Resource


,Resource,Search
0,Account,_language
0,Account,owner
0,Account,identifier
0,Account,period
0,Account,patient
...,...,...
145,VisionPrescription,patient
145,VisionPrescription,datewritten
145,VisionPrescription,_id
145,VisionPrescription,encounter
